In [9]:
#imports

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
#data
from dataset import ONRData

In [10]:
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
from imblearn.over_sampling import SMOTE

# Convert PyTorch dataset to numpy arrays for SMOTE
def dataset_to_numpy(dataset):
    data = []
    labels = []
    for i in range(len(dataset)):
        x, y = dataset[i]  # Assuming (features, label) format
        data.append(x.numpy())  # Convert to NumPy
        labels.append(y)
    return np.array(data), np.array(labels)

# Convert numpy arrays back to a PyTorch dataset
class CustomTensorDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)  # Ensure long dtype for classification
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Load dataset
data = ONRData(label_dtype='int')

# Split into train and test sets
train_data, test_data = random_split(data, [int(0.8*len(data)), len(data)-int(0.8*len(data))])

# Extract features and labels from train_data
X_train, y_train = dataset_to_numpy(train_data)

# Apply SMOTE to balance classes
smote = SMOTE(sampling_strategy='auto', random_state=42)  # 'auto' balances all classes equally
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Convert back to PyTorch Dataset
train_data_smote = CustomTensorDataset(X_resampled, y_resampled)

# Create DataLoaders
train_loader = DataLoader(train_data_smote, batch_size=3, shuffle=True)  # No need for WeightedRandomSampler
test_loader = DataLoader(test_data, batch_size=1, shuffle=False)

# Print class distribution after SMOTE
print("Class distribution after SMOTE:", np.bincount(y_resampled))


23
23
23


ValueError: Found array with dim 4. SMOTE expected <= 2.

In [ ]:
#Define the model
import torch.nn.functional as F

class fNIRS_CNN_LSTM(nn.Module):
    def __init__(self, num_classes=2):
        super(fNIRS_CNN_LSTM, self).__init__()
        
        # Convolutional Layers
        self.conv1 = nn.Conv2d(in_channels=4, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=4, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Dropout layer
        self.dropout = nn.Dropout(0.5)
        
        # LSTM Layer
        self.lstm = nn.LSTM(input_size=32, hidden_size=128, batch_first=True)
        
        # Fully Connected Layers
        self.fc1 = nn.Linear(128, 32)
        self.fc2 = nn.Linear(32, num_classes)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))  # 3x3 Conv
        x = self.pool(x)           # 2x2 MaxPool
        x = F.relu(self.conv2(x))  # 4x4 Conv
        x = self.dropout(x)        # Dropout
        
        # Reshaping for LSTM
        x = x.view(x.size(0), -1, x.size(1))  # (batch, time_steps, features)
        
        x, _ = self.lstm(x)  # LSTM layer
        x = x[:, -1, :]  # Taking the last time step
        
        x = F.relu(self.fc1(x))  # Fully Connected Layer
        x = torch.sigmoid(self.fc2(x))  # Output Layer (Sigmoid Activation)
        
        return x

# Example usage
model = fNIRS_CNN_LSTM()
print(model)


fNIRS_CNN_LSTM(
  (conv1): Conv2d(4, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 32, kernel_size=(4, 4), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (lstm): LSTM(32, 128, batch_first=True)
  (fc1): Linear(in_features=128, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=2, bias=True)
)


In [ ]:
#Hyperparameters
num_epochs = 10
num_classes = 2
batch_size = 64
lr = 0.0001

#Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


#Define the loss function and optimizer

# Loss function
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
import torch
import torch.nn.functional as F
from torchmetrics.classification import Accuracy, Precision, Recall, F1Score, ConfusionMatrix
from tqdm import tqdm  # For progress bar

# Define binary classification metrics
accuracy = Accuracy(task="binary").to(device)
precision = Precision(task="binary").to(device)
recall = Recall(task="binary").to(device)
f1 = F1Score(task="binary").to(device)
conf_matrix = ConfusionMatrix(task="binary", num_classes=2).to(device)

# Move model to device
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    train_acc = 0.0
    train_steps = 0

    # Training loop with progress bar
    loop = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]", leave=True)
    
    for images, labels in loop:
        images = images.to(device)
        labels = labels.to(device).float()  # Ensure labels are float for BCEWithLogitsLoss

        # Forward pass
        outputs = model(images)
        
        #one hot encoding
        labels = F.one_hot(labels.to(torch.int64), num_classes=2).to(torch.float32)
        
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        train_acc += accuracy(outputs.sigmoid(), labels)  # Apply sigmoid for probability
        train_steps += 1

        # Update progress bar
        loop.set_postfix(loss=running_loss / train_steps, acc=train_acc.item() / train_steps)

    print(f"Epoch {epoch+1}, Loss: {running_loss/train_steps:.4f}, Train Accuracy: {train_acc.item()/train_steps:.4f}")

    # ======================= EVALUATION =======================
    model.eval()  # Set model to evaluation mode
    test_acc = 0.0
    test_prec = 0.0
    test_rec = 0.0
    test_f1 = 0.0
    total_samples = 0
    conf_matrix_sum = torch.zeros((2, 2)).to(device)

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device).float()

            outputs = model(images).squeeze(1)
            labels = F.one_hot(labels.to(torch.int64), num_classes=2).to(torch.float32)
            preds = (outputs.sigmoid() > 0.5).float()  # Convert logits to binary predictions

            test_acc += accuracy(preds, labels)
            test_prec += precision(preds, labels)
            test_rec += recall(preds, labels)
            test_f1 += f1(preds, labels)
            conf_matrix_sum += conf_matrix(preds, labels)

            total_samples += labels.size(0)

    # Compute mean metrics
    test_acc /= len(test_loader)
    test_prec /= len(test_loader)
    test_rec /= len(test_loader)
    test_f1 /= len(test_loader)

    print(f"Test Accuracy: {test_acc.item():.4f}")
    print(f"Precision: {test_prec.item():.4f}, Recall: {test_rec.item():.4f}, F1-score: {test_f1.item():.4f}")
    print("Confusion Matrix:\n", conf_matrix_sum.cpu().numpy())


Epoch [1/10]: 100%|██████████| 1175/1175 [00:05<00:00, 201.53it/s, acc=0.5, loss=0.693]


Epoch 1, Loss: 0.6930, Train Accuracy: 0.5000
Test Accuracy: 0.5000
Precision: 0.5000, Recall: 1.0000, F1-score: 0.6667
Confusion Matrix:
 [[  0. 881.]
 [  0. 881.]]


Epoch [2/10]: 100%|██████████| 1175/1175 [00:06<00:00, 186.07it/s, acc=0.5, loss=0.693]


Epoch 2, Loss: 0.6927, Train Accuracy: 0.5000
Test Accuracy: 0.5000
Precision: 0.5000, Recall: 1.0000, F1-score: 0.6667
Confusion Matrix:
 [[  0. 881.]
 [  0. 881.]]


Epoch [3/10]: 100%|██████████| 1175/1175 [00:06<00:00, 192.93it/s, acc=0.5, loss=0.691]


...

Epoch 8, Loss: 0.5514, Train Accuracy: 0.5000
Test Accuracy: 0.5000
Precision: 0.5000, Recall: 1.0000, F1-score: 0.6667
Confusion Matrix:
 [[  0. 881.]
 [  0. 881.]]


Epoch [9/10]: 100%|██████████| 1175/1175 [00:06<00:00, 195.21it/s, acc=0.5, loss=0.512]


Epoch 9, Loss: 0.5116, Train Accuracy: 0.5000
Test Accuracy: 0.5000
Precision: 0.5000, Recall: 1.0000, F1-score: 0.6667
Confusion Matrix:
 [[  0. 881.]
 [  0. 881.]]


Epoch [10/10]:  25%|██▍       | 291/1175 [00:01<00:04, 195.69it/s, acc=0.5, loss=0.476]


KeyboardInterrupt: 